In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('blogtext.csv').sample(4000)
df.head()

,id,gender,age,topic,sign,date,text
490961,2761662,female,26,Education,Libra,"21,July,2004","&nbsp;There is a spider--a big, black,..."
90528,3113729,male,27,Communications-Media,Capricorn,"07,June,2004","...is, like, hands down my fa..."
591711,3991547,male,48,Communications-Media,Capricorn,"22,July,2004",Spammers About 85% of the mail in yo...
41838,888601,female,24,indUnk,Libra,"19,February,2003","Thursday February 20, 2003 Why do all..."
273246,3702804,male,43,Communications-Media,Aries,"07,July,2004",From Brand Autopsy: Being vi...


In [2]:
X=df['text'].astype(str)
Y=pd.get_dummies(df['gender'])
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 100)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train,Y_train, test_size = 0.30, random_state = 100)
print(X_train.shape,Y_train.shape)
print(X_valid.shape,Y_valid.shape)
print(X_test.shape,Y_test.shape)

(1960,) (1960, 2)
(840,) (840, 2)
(1200,) (1200, 2)


In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_tokens = X_train.map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:64])
test_tokens = X_test.apply(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:64])
train_tokens_ids = pd.Series(train_tokens).apply(tokenizer.convert_tokens_to_ids)
test_tokens_ids = pd.Series(test_tokens).apply(tokenizer.convert_tokens_to_ids)
train_tokens_ids.shape

(1960,)

In [4]:
#Do I need this step?
import keras
from keras.preprocessing.sequence import pad_sequences
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=64, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=64, truncating="post", padding="post", dtype="int")

Using TensorFlow backend.


In [5]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
BATCH_SIZE = 64
test_y = np.array(Y_test)
train_y = np.array(Y_train)
train_tokens_tensor = torch.LongTensor(train_tokens_ids)
train_y_tensor = torch.from_numpy(train_y)
test_tokens_tensor = torch.LongTensor(test_tokens_ids)
test_y_tensor = torch.from_numpy(test_y)
train_dataset = TensorDataset(train_tokens_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)
test_dataset = TensorDataset(test_tokens_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
from transformers import AdamW, BertForSequenceClassification, BertModel, BertConfig
EPOCHS=3
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = AdamW(model.parameters(), lr=3e-6)
for epoch_num in range(EPOCHS):
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, labels = [t for t in batch_data]
        outputs = model(input_ids=token_ids, labels=labels[:,0].long())
        loss = outputs[0]
        model.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
outputs